# Training and Testing A Visual Transformer Model
Here we test a visual Transfrom ViT from [An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale](https://arxiv.org/abs/2010.11929)

In [ ]:
import torch
import torchvision as tv
import torchvision.transforms.v2 as v2
from our_datasets import Country_images
from Country_dict import country_dict
from ViT import VisionTransformer

USE_GPU = True
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
#get models
print(f"Using {device} device")

Using cuda device


## Load our Dataset and Create our dataloaders

In [2]:
batch_size = 32
weights = tv.models.ViT_L_32_Weights.DEFAULT
transform = v2.Compose([weights.transforms(), ])

num_classes = len(country_dict)
dataset_path = "data\\compressed_dataset\\"
dataset = Country_images("data\\compressed_dataset\\country.csv",dataset_path,transform=transform)
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset,lengths=[0.7,0.1,0.2])
data_loader_params = {
    'batch_size': batch_size,  # Batch size for data loading
    'num_workers': 10,  # Number of subprocesses to use for data loading
    'persistent_workers': True,  # If True, the data loader will not shutdown the worker processes after a dataset has been consumed once. This allows to maintain the worker dataset instances alive.
    'pin_memory': True,  # If True, the data loader will copy Tensors into CUDA pinned memory before returning them. Useful when using GPU.
    'pin_memory_device': 'cuda' ,  # Specifies the device where the data should be loaded. Commonly set to use the GPU.
}
train_dataloader      = torch.utils.data.DataLoader(train_dataset, **data_loader_params, shuffle=True)
val_dataloader        = torch.utils.data.DataLoader(val_dataset, **data_loader_params, shuffle=True)
test_dataloader       = torch.utils.data.DataLoader(test_dataset, **data_loader_params, shuffle=False,in_order=True)

## Load our model

In [3]:
#hyperparameters:
lr = 0.1
#maximum learning rate we will let our model train in order to train faster at the start
max_lr = 0.1
weight_decay = 0.0
EPOCHS = 100
#end hyperparameters

#model and optimizers
model = tv.models.vit_l_32(weights=weights)
#model = VisionTransformer(num_classes=num_classes)
model.device = device
model.name = "ViT_l_16"
model.path = "Trained_Models\\ViT\\" #where to save our best model
print(model.path)
#redfine our output layer to output our classes
model.heads.head = torch.nn.Linear(in_features=model.heads.head.in_features,out_features=num_classes,bias=True)
print(model)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
#scales the gradients, neccessary for mixed precision data types to properly converge
scaler = torch.amp.GradScaler(device=device)
#change our learning rate based on far we are in training and if we are improving
lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=max_lr, total_steps=EPOCHS*len(train_dataloader))

#added data to our model for ease of use (and to prevent passing so many variables to our training function)


Trained_Models\ViT\
VisionTransformer(
  (conv_proj): Conv2d(3, 1024, kernel_size=(32, 32), stride=(32, 32))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=1024, out_features=4096, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=4096, out_features=1024, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((1024,), eps=1e-06, elementwise_

## Train our Model

In [ ]:
#we will call the function we defined in "Training_Functions.py"
from Training_Functions import TrainModel
model_train = True

if model_train:
    model = model.to(device)
    TrainModel(model,EPOCHS, loss_fn, train_dataloader, val_dataloader, optimizer, lr_scheduler, scaler)
if not model_train:
    checkpoint = torch.load(model.path+model.name+"-best")
    model.load_state_dict(checkpoint)
    model = model.to(device)

EPOCH 1:
  batch 100 loss: 4.360601425170898
  batch 200 loss: 4.322263240814209
  batch 300 loss: 4.3097076416015625
  batch 400 loss: 4.305140495300293
  batch 500 loss: 4.301965236663818
  batch 600 loss: 4.299342632293701
  batch 700 loss: 4.29555082321167
  batch 800 loss: 4.293180465698242
  batch 900 loss: 4.291337013244629
  batch 1000 loss: 4.290902614593506
LOSS train 4.290902614593506 valid 19.809770584106445
Accuracy: 0.0188
EPOCH 2:
  batch 100 loss: 4.323410511016846
  batch 200 loss: 4.295217037200928
  batch 300 loss: 4.291136264801025
  batch 400 loss: 4.286738395690918
  batch 500 loss: 4.2853569984436035
  batch 600 loss: 4.2825093269348145
  batch 700 loss: 4.283159255981445
  batch 800 loss: 4.283255100250244
  batch 900 loss: 4.282806873321533
  batch 1000 loss: 4.28232479095459
LOSS train 4.28232479095459 valid 25.436050415039062
Accuracy: 0.013
EPOCH 3:
  batch 100 loss: 4.3167266845703125
  batch 200 loss: 4.3024115562438965
  batch 300 loss: 4.2920403480529785

## Test our Model

In [ ]:
#we will call the function we defined in "Training_Functions.py"
from Training_Functions import TestModel
TestModel(model, test_dataloader, loss_fn)